# Calculation of volume constraints for Si-composite anodes
Carl Erik L. Foss, Jan Petter Maehlen, IFE, 2021

Acknowledgements: Elkem and Research Council of Norway

In [ ]:
# from IPython.display import display, Math, Latex
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

<img src="SiliconExpansion_fig_001.png">

## Input

## Physical prms

In [ ]:
# Needed input prms

# expansion coeffs
expansion_coeff_si       = 2.8 
expansion_coeff_graphite = 0.1
expansion_coeff_dead     = 0.0

# densities
rho_si = 2.329           # g/ccm
rho_graphite = 2.1       # g/ccm
rho_dead = 1.8           # g/ccm

# max capacities
maxcap_si = 3579.0       # mAh/g
maxcap_graphite = 372.0  # mAh/g
maxcap_dead = 0.0        # mAh/g


In [ ]:
mass_fraction_silicon  = 0.6
mass_fraction_graphite = 0.3
mass_fraction_dead     = 0.1

radius = 0.5*1.5

eps = 1.0  # allowed expansion coef for electrode (1.0 means no room available)

# cut-off capacities
max_cutoff_cap_si = 3579.0       # mAh/g
max_cutoff_cap_graphite = 372.0  # mAh/g
max_cutoff_cap_dead = 0.0        # mAh/g

porosity = 0.5 # initial porosity
porosity_limit = 0.0 # final limit for porosity

In [ ]:
def density_all(densities, mass_fractions, porosity = 0.0, debug=False):
    # density of the combined composite
    
    total_mass = np.sum(mass_fractions)
    
    total_volume = 0.0
    for d,m in zip(densities, mass_fractions):
        if debug:
            print(f"density: {d}\nfraction: {m}")
        total_volume += m/d
    try:
        total_volume = total_volume/(1-porosity)
    except ZeroDivisionError:
        print("ERROR! You have a porosity of 1.0 (meaning your electrode contains nothing)")
        return None
    
    density = total_mass/total_volume
    #print "Density: %f" % (Density)
    return density

In [ ]:
densities = [rho_si, rho_graphite, rho_dead]
mfracs = [mass_fraction_silicon, mass_fraction_graphite, mass_fraction_dead]
density_all(densities, mfracs, porosity = 0.0)

In [ ]:
mass_of_electrode = 2.23/1000  # g
thickness_of_electrode = 12/10000 # cm
area_of_electrode = 0.25 * np.pi * 1.5**2 # cm2 

In [ ]:
def porosity_theoretical(mass, thickness, area, density):
    volume_real = thickness * area
    volume_calc = mass / density
    porosity = 1 - (volume_calc/volume_real)
    
    return porosity

In [ ]:
density = density_all(densities, mfracs, porosity = 0.0)
porosity = porosity_theoretical(mass_of_electrode, thickness_of_electrode, area_of_electrode, density)

In [ ]:
print(porosity)

In [ ]:
# lithiation degree
x = 1.0

In [ ]:
# new porosity
# assuming linear expansion and lithiation degree up to x (x=1 is max)
new_volume_si = expansion_coeff_si * x * volume_si


